In [1]:
import numpy as np
import tensorflow as tf
from tensorflow.keras import Model, layers
from tensorflow.keras.datasets import mnist

In [2]:
num_classes = 10 # total classes (0-9 digits).
num_features = 784 # data features (img shape: 28*28).

learning_rate = 0.001
training_steps = 1000
batch_size = 32
display_step = 100

timesteps = 28 # timesteps.
num_units = 32 # number of neurons for the LSTM layer.

In [3]:
(x_train, y_train), (x_test, y_test) = mnist.load_data()
x_train, x_test = np.array(x_train, np.float32), np.array(x_test, np.float32)
x_train, x_test = x_train.reshape([-1, 28, 28]), x_test.reshape([-1, num_features])
x_train, x_test = x_train / 255., x_test / 255.

In [4]:
train_data = tf.data.Dataset.from_tensor_slices((x_train, y_train))
train_data = train_data.repeat().shuffle(5000).batch(batch_size).prefetch(1)

In [10]:
class LSTM(Model):
    def __init__(self):
        super(LSTM, self).__init__()
        self.lstm_layer = layers.LSTM(units=num_units)
        self.out = layers.Dense(num_classes)
        
    def call(self, x, is_training=False):
        x = self.lstm_layer(x)
        x = self.out(x)
        if not is_training:
            x = tf.nn.softmax(x)
        return x
            
lstm_net = LSTM()

In [6]:
def cross_entropy_loss(x, y):
    y = tf.cast(y, tf.int64)
    loss = tf.nn.sparse_softmax_cross_entropy_with_logits(labels=y, logits=x)
    return tf.reduce_mean(loss)

def accuracy(y_pred, y_true):
    correct_prediction = tf.equal(tf.argmax(y_pred, 1), tf.cast(y_true, tf.int64))
    return tf.reduce_mean(tf.cast(correct_prediction, tf.float32), axis=-1)

In [7]:
optimizer = tf.optimizers.Adam(learning_rate)

In [8]:
def run_optimization(x, y):
    with tf.GradientTape() as g:
        pred = lstm_net(x, is_training=True)
        loss = cross_entropy_loss(pred, y)
        
    gradients = g.gradient(loss, lstm_net.trainable_variables)
    optimizer.apply_gradients(zip(gradients, lstm_net.trainable_variables))

In [11]:
for step, (batch_x, batch_y) in enumerate(train_data.take(training_steps), 1):
    run_optimization(batch_x, batch_y)
    
    if step % display_step == 0:
        pred = lstm_net(batch_x, is_training=True)
        loss = cross_entropy_loss(pred, batch_y)
        acc = accuracy(pred, batch_y)
        print("step: %i, loss: %f, accuracy: %f" % (step, loss, acc))

step: 100, loss: 1.611787, accuracy: 0.468750
step: 200, loss: 0.940707, accuracy: 0.718750
step: 300, loss: 1.037107, accuracy: 0.718750
step: 400, loss: 0.799975, accuracy: 0.781250
step: 500, loss: 0.931192, accuracy: 0.687500
step: 600, loss: 0.857925, accuracy: 0.750000
step: 700, loss: 0.391519, accuracy: 0.875000
step: 800, loss: 0.600899, accuracy: 0.875000
step: 900, loss: 0.200251, accuracy: 0.937500
step: 1000, loss: 0.223271, accuracy: 0.906250
